In [9]:
import pandas as pd


In [10]:
sap_df = pd.read_csv('exports/beli_konsi_sap.csv')
sc_df = pd.read_csv('exports/beli_konsi_sc.csv')


In [11]:
sap_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5690 entries, 0 to 5689
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   profit_center  5690 non-null   object
 1   document_date  5690 non-null   object
 2   posting_date   5690 non-null   object
 3   assignment     5690 non-null   int64 
 4   gl_account     5690 non-null   int64 
 5   text           46 non-null     object
 6   doc_number     5690 non-null   int64 
 7   doc_type       5690 non-null   object
 8   posting_key    5690 non-null   int64 
 9   amount         5690 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 444.7+ KB


In [12]:
sc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43348 entries, 0 to 43347
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   kode_outlet         43348 non-null  object 
 1   no_transaksi        43348 non-null  object 
 2   kode_obat           43348 non-null  int64  
 3   liph_date           43348 non-null  object 
 4   segmen              43348 non-null  object 
 5   hna                 43348 non-null  int64  
 6   qty_jual            43348 non-null  int64  
 7   disc_margin         43348 non-null  float64
 8   disc_principal      43348 non-null  float64
 9   disc_principal_val  43348 non-null  float64
 10  total_bayar         43348 non-null  int64  
 11  harga_jual          43348 non-null  int64  
 12  harga_srp           43348 non-null  int64  
 13  kode_supplier       43348 non-null  int64  
dtypes: float64(3), int64(7), object(4)
memory usage: 4.6+ MB


In [13]:
# Konversi tanggal
sap_df["document_date"] = pd.to_datetime(sap_df["document_date"])
sc_df["liph_date"] = pd.to_datetime(sc_df["liph_date"])

# Normalisasi kolom
sap_df["outlet"] = sap_df["profit_center"]
sc_df["outlet"] = sc_df["kode_outlet"]

sap_df["tanggal"] = sap_df["document_date"]
sc_df["tanggal"] = sc_df["liph_date"]

# Grouping

sap_df["amount"] = sap_df["amount"].astype(float)
sc_df["total_bayar"] = sc_df["total_bayar"].astype(float)
sap_grouped = sap_df.groupby(["outlet", "tanggal"], as_index=False)["amount"].sum()
sc_grouped = sc_df.groupby(["outlet", "tanggal"], as_index=False)["total_bayar"].sum()


In [14]:
sap_grouped

,outlet,tanggal,amount
0,3150,2025-01-31,-432676.0
1,3150,2025-02-28,334067.0
2,3150,2025-03-31,-1687115.0
3,3150,2025-04-30,-3316933.0
4,3150,2025-05-31,-3194913.0
...,...,...,...
5134,BX52,2025-05-27,171450.0
5135,BX52,2025-05-28,757504.0
5136,BX52,2025-05-29,270863.0
5137,BX52,2025-05-30,167072.0


In [15]:
sc_grouped

,outlet,tanggal,total_bayar
0,BX01,2025-01-01,4698847.0
1,BX01,2025-01-02,6820033.0
2,BX01,2025-01-03,6537741.0
3,BX01,2025-01-04,7389372.0
4,BX01,2025-01-05,3009697.0
...,...,...,...
5129,BX52,2025-05-27,175158.0
5130,BX52,2025-05-28,840829.0
5131,BX52,2025-05-29,300658.0
5132,BX52,2025-05-30,185450.0


In [16]:
validasi = pd.merge(sap_grouped, sc_grouped, on=["outlet", "tanggal"], how="outer")
validasi["selisih"] = validasi["total_bayar"] - validasi["amount"]
validasi["status"] = validasi["selisih"].apply(lambda x: "VALID" if pd.isna(x) or abs(x) < 1 else "TIDAK VALID")

validasi.sort_values(by=["status", "outlet", "tanggal"], ascending=[False, True, True])


,outlet,tanggal,amount,total_bayar,selisih,status
0,3150,2025-01-31,-432676.0,NaN,NaN,VALID
1,3150,2025-02-28,334067.0,NaN,NaN,VALID
2,3150,2025-03-31,-1687115.0,NaN,NaN,VALID
3,3150,2025-04-30,-3316933.0,NaN,NaN,VALID
4,3150,2025-05-31,-3194913.0,NaN,NaN,VALID
...,...,...,...,...,...,...
5134,BX52,2025-05-27,171450.0,175158.0,3708.0,TIDAK VALID
5135,BX52,2025-05-28,757504.0,840829.0,83325.0,TIDAK VALID
5136,BX52,2025-05-29,270863.0,300658.0,29795.0,TIDAK VALID
5137,BX52,2025-05-30,167072.0,185450.0,18378.0,TIDAK VALID
